Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

Данные лежат на google диске по [ссылке](https://drive.google.com/file/d/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ/view?usp=sharing).

In [36]:
!wget 'https://drive.google.com/uc?export=download&id=1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ' -O WA_Fn-UseC_-Telco-Customer-Churn.csv

--2022-02-21 11:12:24--  https://drive.google.com/uc?export=download&id=1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ
Распознаётся drive.google.com (drive.google.com)… 216.58.207.238
Подключение к drive.google.com (drive.google.com)|216.58.207.238|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://doc-08-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/b3bm3b4qt52eci1nuo1k0voggf8r5r6c/1645431075000/14904333240138417226/*/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ?e=download [переход]
Предупреждение: в HTTP маски не поддерживаются.
--2022-02-21 11:12:25--  https://doc-08-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/b3bm3b4qt52eci1nuo1k0voggf8r5r6c/1645431075000/14904333240138417226/*/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ?e=download
Распознаётся doc-08-c0-docs.googleusercontent.com (doc-08-c0-docs.googleusercontent.com)… 216.58.207.225
Подключение к doc-08-c0-docs.googleusercontent.com (doc-08-c0-docs.go

In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [38]:
f = df.gender.loc[df['gender'] == 'Female'].count() 
m = df.gender.loc[df['gender'] == 'Male'].count()
print(f'female = {(f/(f + m))*100}, male = {(m/(f + m))*100}')

female = 49.5243504188556, male = 50.4756495811444


##### 2. Какое количество уникальных значений у поля InternetService?

In [39]:
u = set(df.InternetService.unique())
print("Уникальных значений - ",len(u),"\n", u)


Уникальных значений -  3 
 {'DSL', 'Fiber optic', 'No'}


##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [40]:
df.TotalCharges.describe()

count     7043
unique    6531
top           
freq        11
Name: TotalCharges, dtype: object

In [41]:
print(f'Медиана: {df.TotalCharges.median()}')
print(f'Медиана: {df.TotalCharges.mean():.2f}')
print(f'Среднеквадратичное отклонение: {df.TotalCharges.std():.2f}')

TypeError: 

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

Столбец типа Object с пропусками. Нужно привести данные к числовому формату

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [42]:
df.PhoneService.head()

0     No
1    Yes
2    Yes
3     No
4    Yes
Name: PhoneService, dtype: object

In [43]:
df.PhoneService.replace(to_replace='Yes', value='1', inplace=True)
df.PhoneService.replace(to_replace='No', value='0', inplace=True)
df.PhoneService = pd.to_numeric(df.PhoneService)

In [44]:
df.PhoneService.head()

0    0
1    1
2    1
3    0
4    1
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [45]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan).astype('float32').fillna(0)


In [46]:
df.TotalCharges.describe()

count    7043.000000
mean     2279.734375
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

In [47]:
print(f'Медиана: {df.TotalCharges.median()}')
print(f'Медиана: {df.TotalCharges.mean():.2f}')
print(f'Среднеквадратичное отклонение: {df.TotalCharges.std():.2f}')

Медиана: 1394.550048828125
Медиана: 2279.73
Среднеквадратичное отклонение: 2266.79


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [48]:
df.Churn.head()

0     No
1     No
2    Yes
3     No
4    Yes
Name: Churn, dtype: object

In [49]:
df.Churn.replace(to_replace='Yes', value='1', inplace=True)
df.Churn.replace(to_replace='No', value='0', inplace=True)
df.Churn = pd.to_numeric(df.Churn)
df.Churn.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [50]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head()

,StreamingMovies,StreamingTV,TechSupport
0,No,No,No
1,No,No,No
2,No,No,No
3,No,No,Yes
4,No,No,No


In [51]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']] = df[['StreamingMovies', 'StreamingTV', 'TechSupport']].replace(to_replace={'Yes': 1, 'No': 0, 'No internet service': 0})
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head()

,StreamingMovies,StreamingTV,TechSupport
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,1
4,0,0,0


##### 8. Заполните пропуски в поле PhoneService значением 0

In [52]:
df.PhoneService.describe()

count    7043.000000
mean        0.903166
std         0.295752
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: PhoneService, dtype: float64

In [53]:
df.PhoneService.fillna(0)
df.PhoneService.describe()

count    7043.000000
mean        0.903166
std         0.295752
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: PhoneService, dtype: float64

##### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [54]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df.drop(df.columns.difference(columns), 1, inplace=True)
df.head(3)

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
0,Female,1,0,0,0,0,29.850000,0
1,Male,34,1,0,0,0,1889.500000,0
2,Male,2,1,0,0,0,108.150002,1


##### 9. Разделите датасет на тренировочную и тестовую выборку.

In [55]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X_train, X_test, y_train, y_test = train_test_split(df, df[target], test_size=0.33, shuffle=True, random_state=0)


In [56]:
X_test

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
2200,Female,19,1,0,1,0,1045.250000,0
4627,Male,60,1,1,1,1,7049.500000,0
3225,Female,13,1,0,1,1,923.849976,0
2828,Male,1,1,0,0,0,20.450001,0
3768,Female,55,1,0,0,0,4266.399902,0
...,...,...,...,...,...,...,...,...
5407,Female,63,1,1,0,0,4014.199951,0
3909,Male,11,1,0,0,0,223.149994,0
6747,Female,52,0,0,0,0,1958.949951,0
2774,Male,6,1,1,0,0,288.350006,0


In [57]:
X_train

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
2337,Female,56,1,0,1,1,5510.649902,1
2560,Male,46,1,0,0,1,4154.799805,1
4725,Male,1,1,1,0,0,62.049999,1
4278,Female,48,1,0,1,1,4692.950195,0
3592,Female,1,1,0,0,0,45.650002,1
...,...,...,...,...,...,...,...,...
4931,Male,15,1,1,1,1,1539.800049,0
3264,Female,10,1,1,1,0,964.349976,0
1653,Male,58,1,0,0,0,1185.949951,0
2607,Male,1,1,0,0,0,69.750000,1


In [58]:
y_test

2200    0
4627    0
3225    0
2828    0
3768    0
       ..
5407    0
3909    0
6747    0
2774    0
1177    0
Name: Churn, Length: 2325, dtype: int64

In [59]:
y_train

2337    1
2560    1
4725    1
4278    0
3592    1
       ..
4931    0
3264    0
1653    0
2607    1
2732    0
Name: Churn, Length: 4718, dtype: int64

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [60]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [61]:
gender.fit_transform(X_train)


,gender_Female,gender_Male
2337,1,0
2560,0,1
4725,0,1
4278,1,0
3592,1,0
...,...,...
4931,0,1
3264,1,0
1653,0,1
2607,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [62]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

Стандартизация признаков полезна в случае линейной модели

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [63]:
TotalCharges = Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
])

Объединение всех "кубиков" очень легко сделать таким образом

In [64]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [65]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [66]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [67]:
pipeline.predict(X_test)


array([0, 0, 0, ..., 0, 0, 0])

In [68]:
len(X_test), len(pipeline.predict(X_test))

(2325, 2325)

##### 15. Посчитайте метрики качества получившейся модели (roc_auc)

In [69]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score(y_test, pipeline.predict(X_test)), log_loss(y_test, pipeline.predict(X_test))


(0.6562576229970575, 8.482506506535584)

### Сохраним наш пайплайн

In [70]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)